In [1]:
from seisLM.data_pipeline.foreshock_aftershock_dataset import (
  create_foreshock_aftershock_datasets
)
from seisLM.data_pipeline.foreshock_aftershock_dataloaders import (
  prepare_foreshock_aftershock_dataloaders)
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
NUM_CLASSES = 4
BATCH_SIZE = 8
EVENT_SPLIT_METHOD = 'temporal'
COMPONENT_ORDER = 'ZNE'
DIMENSION_ORDER = 'NCW'

In [3]:
loaders = prepare_foreshock_aftershock_dataloaders(
  num_classes=NUM_CLASSES,
  batch_size=BATCH_SIZE,
  event_split_method=EVENT_SPLIT_METHOD,
  component_order=COMPONENT_ORDER,
  standardize=True,
)


Seed set to 42
Seed set to 42


In [4]:
import json
import ml_collections
import torch
from seisLM.model.task_specific.foreshock_aftershock_models import ShockClassifierLit #Wav2Vec2ForSequenceClassification

def get_seisLM_config():

  config_path = '/scicore/home/dokman0000/liu0003/projects/seisLM/seisLM/configs/foreshock_aftershock/seisLM_shock_classifier.json'
  with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
  config = ml_collections.ConfigDict(config)

  config.model_args.num_classes = NUM_CLASSES
  config.model_args.use_weighted_layer_sum = True
  return config

seisLM_config = get_seisLM_config()

models = {}

models['seisLM'] = ShockClassifierLit(
  model_name='Wav2Vec2ForSequenceClassification',
  model_config=seisLM_config.model_args,
  training_config=seisLM_config.trainer_args,
  ).model



In [5]:
def get_conv1d_config():
  config_path = '/scicore/home/dokman0000/liu0003/projects/seisLM/seisLM/configs/foreshock_aftershock/conv1d_shock_classifier.json'
  with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
  config = ml_collections.ConfigDict(config)
  config.model_args.num_classes = NUM_CLASSES
  return config

conv1d_config = get_conv1d_config()

models['cnn'] = ShockClassifierLit(
  model_name='Conv1DShockClassifier',
  model_config=conv1d_config.model_args,
  training_config=conv1d_config.trainer_args,
  ).model

In [6]:
waveforms, labels = next(iter(loaders['train']))


key = 'cnn'
# key = 'seisLM'
model = models[key]


logits = models[key](waveforms)
loss = torch.nn.functional.cross_entropy(logits, labels)
predicted_labels = torch.argmax(logits, 1)

logits

tensor([[ 5.0726e-02, -1.3623e-02, -3.3230e-02,  8.9958e-03],
        [ 7.8597e-02, -2.4604e-02, -1.4668e-03,  6.0729e-03],
        [ 4.0387e-02, -3.1227e-02, -1.6944e-02,  1.5368e-02],
        [ 8.1868e-02, -3.7683e-02, -1.8850e-02, -1.9350e-02],
        [ 9.9852e-02, -5.3649e-02, -2.4900e-02,  1.2286e-02],
        [ 5.6987e-02, -3.0472e-02, -1.6659e-02,  1.2778e-02],
        [-1.1615e-01, -1.8889e-02,  2.5478e-01,  3.8585e-01],
        [ 1.2838e-02, -1.0144e-05, -5.2672e-03,  6.0781e-02]],
       grad_fn=<AddmmBackward0>)

In [7]:

# for key, model in models.items():

# key = 'cnn'
key = 'seisLM'
model = models[key]


logits = models[key](waveforms)
loss = torch.nn.functional.cross_entropy(logits, labels)
predicted_labels = torch.argmax(logits, 1)


In [11]:
loss




tensor(1.3348, grad_fn=<NllLossBackward0>)

In [9]:
torch.nn.functional.softmax(models['seisLM'].layer_weights)




/tmp/ipykernel_21553/602353954.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.nn.functional.softmax(models['seisLM'].layer_weights)


tensor([4.5388e-05, 4.5388e-05, 4.5388e-05, 4.5388e-05, 4.5388e-05, 4.5388e-05,
        9.9973e-01], grad_fn=<SoftmaxBackward0>)

In [8]:
import torch

from torch import nn



m = nn.Dropout1d(p=0.5)
input = torch.randn(1, 16, 240)

output = m(input)
output


tensor([[[ 2.4699, -0.1249, -2.7283,  ..., -2.1645,  3.3354,  3.9700],
         [-3.2628, -0.1869,  2.3210,  ...,  3.3528, -1.5258, -3.6013],
         [-2.6599, -1.1798, -2.8883,  ..., -0.7350,  1.5646,  2.5581],
         ...,
         [-3.3682,  0.1104,  0.4925,  ...,  1.8187,  0.0572,  2.3055],
         [ 0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
         [-0.0841, -2.2457, -0.5701,  ..., -1.4444, -2.9667, -3.3141]]])

In [12]:
output[0][-2].shape


torch.Size([240])

In [ ]:
output